<a href="https://colab.research.google.com/github/baovin/ML_homework/blob/main/Inclass_Lab02_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
class environment:
  def __init__(self, grid_height, grid_width):
    """
    The map is initialized with height and width are varible of your choice
    start: List of location where you step in, you get to the corresponding location in list 'end'
           For example:
              if you step in location start[3] then you get to new location end[3] then obtain the reward value of reward[3]
    reward: List of reward value where you move from a location in 'start' list to the corresponding location in 'end' list
    """
    self.height = grid_height
    self.width = grid_width
    self.start = []
    self.end = []
    self.reward = []
    self.map = np.array([i for i in range(grid_height * grid_width)])
    self.action_space = [0,1,2,3]

  def get_Map(self):
    print(self.map.reshape([self.width, self.height]))

  def get_NumState(self):
    return self.height * self.width

  def map_Designate(self, start_cell, end_cell, reward):
    self.start.append(start_cell)
    self.end.append(end_cell)
    self.reward.append(reward)

  def get_Observation(self, location, action):
    # Default reward = 0
    reward = 0
    new_location = 0
    # Action: UP: 0, DOWN: 1, LEFT: 2, RIGHT: 3
    # Actions that get the agent out of the map, result in no change at all
    if action == 0: #UP
      if location - self.width < 0:
        new_location = location
      else:
        new_location = location - self.width

    elif action == 1: #DOWN
      if location + self.width > self.height * self.width - 1:
        new_location = location
      else:
        new_location = location + self.width

    elif action == 2: #LEFT
      if location % self.width == 0:
        new_location = location
      else:
        new_location = location - 1

    elif action == 3: #RIGHT
      if (location + 1) % self.width == 0:
        new_location = location
      else:
        new_location = location + 1

    # If the agent is at special locations, immediately moves to corresponding destinations, gain reward
    if new_location in self.start:
      idx = self.start.index(new_location)
      new_location = self.end[idx]
      reward = self.reward[idx]

    return new_location, self.action_space, reward


class MAB_agent:
  def __init__(self, envir, init_location):
    # Trace the reward
    self.reward_trace = []
    # initialize the first location
    self.location_now = init_location
    # TODO: implement other features to the agent so it can perform MAB algorithm
    self.lastAction = None
    self.lastState = None
    self.value_table = {}    # format: {state : {action : [value, count]}}
    for state in range(envir.width*envir.height):
      self.value_table[state] = {}
      for action in envir.action_space:
        self.value_table[state][action] = [0,0]

  def get_TotalReward(self):
    return np.sum(self.reward_trace)

  # Running in Simulator
  def getAction(self, observation):
    self.location_now, action_space, pre_reward = observation
    # NOTICE: the first observation is (init_location, [0,1,2,3], None)
    # You should process the 'None'
    if pre_reward is None:
      action = np.random.choice(action_space)
    else:
      self.reward_trace.append(pre_reward)
      # Updating with incremental average over reward samples
      value = self.value_table[self.lastState][self.lastAction][0]
      count = self.value_table[self.lastState][self.lastAction][1]

      count += 1
      value += (1/count) * (pre_reward - value)

      self.value_table[self.lastState][self.lastAction][0] = value
      self.value_table[self.lastState][self.lastAction][1] = count

      # # get action
      # state_dict = self.value_table[self.location_now]
      # max = max([])
      # action, max_value = max(state_dict.items(), key=lambda x: x[1][0])

      # get action
      state_dict = self.value_table[self.location_now]
      # action, max_value = max(state_dict.items(), key=lambda x: x[1][0])
      list_value = [f[0] for f in state_dict.values()]
      max_value = max(list_value)
      # find all actions that have maximum value
      max_actions = [index for index, value in enumerate(list_value) if value == max_value]
      if len(max_actions) == 1:
        action = max_actions[0]
      else:
        # find among maximum actions which has already done
        freq_actions = [state_dict[a][1] for a in max_actions]
        max_freq_actions = [index for index, value in enumerate(freq_actions) if value == max(freq_actions)]
        if len(max_freq_actions) == 1:
          index = max_freq_actions[0]
        else:
          # there are multiple actions that have same value, same frequent, use random
          index = np.random.choice(max_freq_actions)
        action = max_actions[index]


    self.lastState = self.location_now
    self.lastAction = action
    # Assert valid action
    assert action in action_space, "INVALID action taken"
    return action


class MABe_agent(MAB_agent):
  def __init__(self, envir, init_location, epsilon):
    super(MABe_agent, self).__init__(envir, init_location)
    self.epsilon = epsilon


  # Override
  def getAction(self, observation):
    self.location_now, action_space, pre_reward = observation
    # NOTICE: the first observation is (init_location, [0,1,2,3], None)
    # You should process the 'None'

    toss = np.random.rand()
    if pre_reward is None or toss < self.epsilon:
      action = np.random.choice(action_space)
    else:
      self.reward_trace.append(pre_reward)
      # Updating with incremental average over reward samples
      value = self.value_table[self.lastState][self.lastAction][0]
      count = self.value_table[self.lastState][self.lastAction][1]

      count += 1
      value += (1/count) * (pre_reward - value)

      self.value_table[self.lastState][self.lastAction][0] = value
      self.value_table[self.lastState][self.lastAction][1] = count

      # get action
      state_dict = self.value_table[self.location_now]
      # action, max_value = max(state_dict.items(), key=lambda x: x[1][0])
      list_value = [f[0] for f in state_dict.values()]
      max_value = max(list_value)
      # find all actions that have maximum value
      max_actions = [index for index, value in enumerate(list_value) if value == max_value]
      if len(max_actions) == 1:
        action = max_actions[0]
      else:
        # find among maximum actions which has already done
        freq_actions = [state_dict[a][1] for a in max_actions]
        min_freq_actions = [index for index, value in enumerate(freq_actions) if value == min(freq_actions)]
        if len(min_freq_actions) == 1:
          index = min_freq_actions[0]
        else:
          # there are multiple actions that have same value, same frequent, use random
          index = np.random.choice(min_freq_actions)
        action = max_actions[index]



    self.lastState = self.location_now
    self.lastAction = action
    # Assert valid action
    assert action in action_space, "INVALID action taken"
    return action

In [ ]:
# Create environment
Envir = environment(8,8)
Envir.map_Designate(17,56,-15)
Envir.map_Designate(18,56,-15)
Envir.map_Designate(19,56,-15)
Envir.map_Designate(21,56,-15)
Envir.map_Designate(25,56,-15)
Envir.map_Designate(33,56,-15)
Envir.map_Designate(41,56,-15)
Envir.map_Designate(42,56,-15)
Envir.map_Designate(43,56,-15)
Envir.map_Designate(46,56,-15)
Envir.map_Designate(47,56,-15)
Envir.map_Designate(47,56,-15)
Envir.map_Designate(15,56,+15)
Envir.map_Designate(1,10,+5)
Envir.map_Designate(26,56,+20)

In [ ]:
Envir.get_Map()

###Q-learning

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Implement your agent
class Q_agent(MAB_agent):
  def __init__(self, envir, init_location, learning_rate, discount_factor, epsilon):
    super(Q_agent, self).__init__(envir, init_location)
    self.Q_table = {}   # format: {state : {action : [value, count]}}
    for state in range(envir.width*envir.height):
      self.Q_table[state] = {}
      for action in envir.action_space:
        self.Q_table[state][action] = [0,0]
    self.alpha = learning_rate          # learning rate
    self.gamma = discount_factor        # discount factor
    self.epsilon = epsilon

  # Overide method
  def getAction(self, observation):
    self.location_now, action_space, pre_reward = observation
    # NOTICE: the first observation is (NONE, [0,1,2,3], None)
    # You should process the 'None' value
    if pre_reward is None:
      # randomly select among action space
      ########## YOUR CODE HERE ############
    else:
      # get action
      ########## YOUR CODE HERE ############



      # update Q-table
      ########## YOUR CODE HERE ############



    self.lastState = self.location_now
    self.lastAction = action
    # Assert valid action
    assert action in action_space, "INVALID action taken"
    return action

In [ ]:
init_location=0
dummy_q_agent = Q_agent(envir=Envir, init_location=init_location, learning_rate=0.8, discount_factor=0.8, epsilon=0.3)

num_iter = 10000

log_freq = 1000
Data_plot1 = []

for i in range(num_iter):
  if i == 0:
    env_observation = (init_location, Envir.action_space, None)
  else:
    env_observation = Envir.get_Observation(location=dummy_q_agent.location_now, action=chosen_action)

  chosen_action = dummy_q_agent.getAction(observation=env_observation)

  if (i + 1) % log_freq == 0:
    aver = np.mean(dummy_q_agent.reward_trace)
    Data_plot1.append(aver)
    print('iter: ' + str(i + 1) + '\t Total reward: ' + str(dummy_q_agent.get_TotalReward()) + '\t Average: ' + str(aver))

In [ ]:
Y_Cord = ["A", "B", "C", "D", "E", "F", "G", "H"]
X_Cord = ["1", "2", "3", "4", "5", "6", "7", "8"]

# Map = freq_list.reshape([Envir.width, Envir.height])
Map = np.zeros([Envir.width, Envir.height])
for i in range(Envir.height):
  for j in range(Envir.width):
    state = i * Envir.width + j
    if state in dummy_q_agent.Q_table.keys():
      list_freq = [f[1] for f in dummy_q_agent.Q_table[state].values()]
      Map[i][j] = sum(list_freq)
    else:
      Map[i][j] = 0

fig, ax = plt.subplots(figsize=(8,8))
im = ax.imshow(Map, cmap="summer")

# We want to show all ticks...
ax.set_xticks(np.arange(len(X_Cord)))
ax.set_yticks(np.arange(len(Y_Cord)))
# ... and label them with the respective list entries
ax.set_xticklabels(X_Cord)
ax.set_yticklabels(Y_Cord)

# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=0, ha="right",
         rotation_mode="anchor")

# Loop over data dimensions and create text annotations.
for i in range(len(Y_Cord)):
    for j in range(len(X_Cord)):
        text = ax.text(j, i, Map[i, j],
                       ha="center", va="center", color="k")

ax.set_title("Number of actions taken, " + str(num_iter) + " time-steps", fontdict={'size':16})
# ax.set_xlabel("Alpha = " + str(learning_rate) + " Gamma = " + str(discount_factor))
fig.tight_layout()
plt.show()

In [ ]:
Y_Cord = ["A", "B", "C", "D", "E", "F", "G", "H"]
X_Cord = ["1", "2", "3", "4", "5", "6", "7", "8"]
# agent = dummyAgent_e

# Map = freq_list.reshape([Envir.width, Envir.height])
Map = np.zeros([Envir.width, Envir.height])
for i in range(Envir.height):
  for j in range(Envir.width):
    state = i * Envir.width + j
    if state in dummy_q_agent.Q_table.keys():
      list_value = [round(f[0],1) for f in dummy_q_agent.Q_table[state].values()]
      Map[i][j] = round(sum(list_value),1)
    else:
      Map[i][j] = 0

fig, ax = plt.subplots(figsize=(6,6))
im = ax.imshow(Map, cmap="summer")

# We want to show all ticks...
ax.set_xticks(np.arange(len(X_Cord)))
ax.set_yticks(np.arange(len(Y_Cord)))
# ... and label them with the respective list entries
ax.set_xticklabels(X_Cord)
ax.set_yticklabels(Y_Cord)

# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=0, ha="right",
         rotation_mode="anchor")

# Loop over data dimensions and create text annotations.
for i in range(len(Y_Cord)):
    for j in range(len(X_Cord)):
        text = ax.text(j, i, Map[i, j],
                       ha="center", va="center", color="k")

ax.set_title("State value after " + str(num_iter) + " time-steps", fontdict={'size':16})
# ax.set_xlabel("Alpha = " + str(learning_rate) + " Gamma = " + str(discount_factor))
fig.tight_layout()
plt.show()

In [ ]:
Y_Cord = ["A", "B", "C", "D", "E", "F", "G", "H"]
X_Cord = ["1", "2", "3", "4", "5", "6", "7", "8"]


def decode_action(action):
  if action == 0: #UP
    return "\u2191"
  if action == 1: #DOWN
    return "\u2193"
  if action == 2: #LEFT
    return "\u2190"
  return "\u2192" #RIGHT

Map = np.zeros([Envir.width, Envir.height])
Map_labels = [['' for _ in range(Envir.width)] for _ in range(Envir.height)]
for i in range(Envir.height):
  for j in range(Envir.width):
    state = i * Envir.width + j
    if state in dummy_q_agent.Q_table.keys():
      list_value = [f[0] for f in dummy_q_agent.Q_table[state].values()]
      max_value = max(list_value)
      actions = [index for index, value in enumerate(list_value) if value == max_value]
      if len(actions) > 1:
        Map_labels[i][j] = ''
        Map[i][j] = 1
      else:
        Map_labels[i][j] = decode_action(actions[0])
        Map[i][j] = 0

fig, ax = plt.subplots(figsize=(6,6))
im = ax.imshow(Map,cmap='summer')

# We want to show all ticks...
ax.set_xticks(np.arange(len(X_Cord)))
ax.set_yticks(np.arange(len(Y_Cord)))
# ... and label them with the respective list entries
ax.set_xticklabels(X_Cord)
ax.set_yticklabels(Y_Cord)

# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=0, ha="right",
         rotation_mode="anchor")

# Loop over data dimensions and create text annotations.
for i in range(len(Y_Cord)):
    for j in range(len(X_Cord)):
        text = ax.text(j, i, Map_labels[i][j],
                       ha="center", va="center", color="k",fontsize=20)

ax.set_title("Directions after " + str(num_iter) + " time-steps", fontdict={'size':16})
# ax.set_xlabel("Alpha = " + str(learning_rate) + " Gamma = " + str(discount_factor))
fig.tight_layout()
plt.show()

In [ ]:
Envir.get_Map()

###Expected SARSA

In [ ]:
# Implement your agent
class eSARSA_agent(MAB_agent):
  def __init__(self, envir, init_location, learning_rate, discount_factor, epsilon):
    super(eSARSA_agent, self).__init__(envir, init_location)
    # TODO: initialize your Q table
    self.Q_table = {}   # format: {state : {action : [value, count]}}
    for state in range(envir.width*envir.height):
      self.Q_table[state] = {}
      for action in envir.action_space:
        self.Q_table[state][action] = [0,0]
    self.alpha = learning_rate          # learning rate
    self.gamma = discount_factor        # discount factor
    self.epsilon = epsilon

  # Overide method
   def getAction(self, observation):
    self.location_now, action_space, pre_reward = observation
    # NOTICE: the first observation is (NONE, [0,1,2,3], None)
    # You should process the 'None' value
    if pre_reward is None:
      # randomly select among action space
      ########## YOUR CODE HERE ############
    else:
      # get action
      ########## YOUR CODE HERE ############



      # update Q-table
      ########## YOUR CODE HERE ############

    self.lastState = self.location_now
    self.lastAction = action
    # Assert valid action
    assert action in action_space, "INVALID action taken"
    return action

In [ ]:
init_location=0
dummy_esarsa_agent = eSARSA_agent(envir=Envir, init_location=init_location, learning_rate=0.8, discount_factor=0.8, epsilon=0.3)

num_iter = 10000

log_freq = 1000
Data_plot1 = []

for i in range(num_iter):
  if i == 0:
    env_observation = (init_location, Envir.action_space, None)
  else:
    env_observation = Envir.get_Observation(location=dummy_esarsa_agent.location_now, action=chosen_action)

  chosen_action = dummy_esarsa_agent.getAction(observation=env_observation)

  if (i + 1) % log_freq == 0:
    aver = np.mean(dummy_esarsa_agent.reward_trace)
    Data_plot1.append(aver)
    print('iter: ' + str(i + 1) + '\t Total reward: ' + str(dummy_esarsa_agent.get_TotalReward()) + '\t Average: ' + str(aver))

In [ ]:
Y_Cord = ["A", "B", "C", "D", "E", "F", "G", "H"]
X_Cord = ["1", "2", "3", "4", "5", "6", "7", "8"]
# agent = dummyAgent_e

# Map = freq_list.reshape([Envir.width, Envir.height])
Map = np.zeros([Envir.width, Envir.height])
for i in range(Envir.height):
  for j in range(Envir.width):
    state = i * Envir.width + j
    if state in dummy_esarsa_agent.Q_table.keys():
      list_value = [round(f[0],1) for f in dummy_esarsa_agent.Q_table[state].values()]
      Map[i][j] = round(sum(list_value),1)
    else:
      Map[i][j] = 0

fig, ax = plt.subplots(figsize=(6,6))
im = ax.imshow(Map, cmap="summer")

# We want to show all ticks...
ax.set_xticks(np.arange(len(X_Cord)))
ax.set_yticks(np.arange(len(Y_Cord)))
# ... and label them with the respective list entries
ax.set_xticklabels(X_Cord)
ax.set_yticklabels(Y_Cord)

# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=0, ha="right",
         rotation_mode="anchor")

# Loop over data dimensions and create text annotations.
for i in range(len(Y_Cord)):
    for j in range(len(X_Cord)):
        text = ax.text(j, i, Map[i, j],
                       ha="center", va="center", color="k")

ax.set_title("State value after " + str(num_iter) + " time-steps", fontdict={'size':16})
# ax.set_xlabel("Alpha = " + str(learning_rate) + " Gamma = " + str(discount_factor))
fig.tight_layout()
plt.show()

In [ ]:
Y_Cord = ["A", "B", "C", "D", "E", "F", "G", "H"]
X_Cord = ["1", "2", "3", "4", "5", "6", "7", "8"]


def decode_action(action):
  if action == 0: #UP
    return "\u2191"
  if action == 1: #DOWN
    return "\u2193"
  if action == 2: #LEFT
    return "\u2190"
  return "\u2192" #RIGHT

Map = np.zeros([Envir.width, Envir.height])
Map_labels = [['' for _ in range(Envir.width)] for _ in range(Envir.height)]
for i in range(Envir.height):
  for j in range(Envir.width):
    state = i * Envir.width + j
    if state in dummy_esarsa_agent.Q_table.keys():
      list_value = [f[0] for f in dummy_esarsa_agent.Q_table[state].values()]
      max_value = max(list_value)
      actions = [index for index, value in enumerate(list_value) if value == max_value]
      if len(actions) > 1:
        Map_labels[i][j] = ''
        Map[i][j] = 1
      else:
        Map_labels[i][j] = decode_action(actions[0])
        Map[i][j] = 0

fig, ax = plt.subplots(figsize=(6,6))
im = ax.imshow(Map,cmap='summer')

# We want to show all ticks...
ax.set_xticks(np.arange(len(X_Cord)))
ax.set_yticks(np.arange(len(Y_Cord)))
# ... and label them with the respective list entries
ax.set_xticklabels(X_Cord)
ax.set_yticklabels(Y_Cord)

# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=0, ha="right",
         rotation_mode="anchor")

# Loop over data dimensions and create text annotations.
for i in range(len(Y_Cord)):
    for j in range(len(X_Cord)):
        text = ax.text(j, i, Map_labels[i][j],
                       ha="center", va="center", color="k",fontsize=20)

ax.set_title("Directions after " + str(num_iter) + " time-steps", fontdict={'size':16})
# ax.set_xlabel("Alpha = " + str(learning_rate) + " Gamma = " + str(discount_factor))
fig.tight_layout()
plt.show()